In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand
158,AFG,Asia,Afghanistan,2020-06-16,25527.0,761.0,478.0,7.0,655.743,19.549,...,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50
258,ALB,Europe,Albania,2020-06-16,1590.0,69.0,36.0,0.0,552.505,23.977,...,13.188,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89
422,DZA,Africa,Algeria,2020-06-16,11031.0,112.0,777.0,10.0,251.556,2.554,...,6.211,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-06-15,France,157732.0,0.0,29432.0,0.0,0.0,0.0
0,2020-06-15,China,84422.0,0.0,4650.0,0.0,0.0,0.0
0,2020-06-15,Italy,237457.0,0.0,34442.0,0.0,0.0,0.0
0,2020-06-15,Spain,244373.0,0.0,27146.0,0.0,0.0,0.0
0,2020-06-15,United States,2121724.0,0.0,116645.0,0.0,0.0,0.0
0,2020-06-15,World,8045552.0,0.0,438076.0,0.0,0.0,0.0
0,2020-06-15,United Kingdom,297671.0,0.0,41926.0,0.0,0.0,0.0
0,2020-06-15,Germany,186798.0,0.0,8808.0,0.0,0.0,0.0
0,2020-06-15,Iran,190081.0,0.0,8911.0,0.0,0.0,0.0
0,2020-06-15,Turkey,179593.0,0.0,4820.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-06-16,France,157732.0,157372.0,29432.0,29436.0,360.0,-4.0
0,2020-06-16,China,84422.0,84378.0,4650.0,4638.0,44.0,12.0
0,2020-06-16,Italy,237457.0,237290.0,34442.0,34371.0,167.0,71.0
0,2020-06-15,Spain,244373.0,244109.0,27146.0,27136.0,264.0,10.0
0,2020-06-16,United States,2121724.0,2114026.0,116645.0,116127.0,7698.0,518.0
0,2020-06-16,World,8045552.0,8000847.0,438076.0,436632.0,44705.0,1444.0
0,2020-06-16,United Kingdom,297671.0,296857.0,41926.0,41736.0,814.0,190.0
0,2020-06-16,Germany,186798.0,186839.0,8808.0,8800.0,-41.0,8.0
0,2020-06-16,Iran,190081.0,189876.0,8911.0,8950.0,205.0,-39.0
0,2020-06-16,Turkey,179593.0,179831.0,4820.0,4825.0,-238.0,-5.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")